# Decision Tree Classifier

Trees are a popular class of algorithm in Machine Learning. In this notebook we build a simple Decision Tree Classifier using `scikit-learn` to show that they can be executed homomorphically using Concrete.

Converting a tree working over quantized data to its FHE equivalent takes only a few lines of code thanks to Concrete ML.

Let's dive in!

## The use case

The use case is a spam classification task from OpenML you can find here: https://www.openml.org/d/44

Some pre-extracted features (like some word frequencies) are provided as well as a class - `0` for a normal e-mail and `1` for spam - for 4601 samples.

Let's first get the data-set.

In [1]:
import time
import numpy
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from concrete.ml.sklearn import DecisionTreeClassifier as ConcreteDecisionTreeClassifier
from sklearn.metrics import average_precision_score, confusion_matrix

In [2]:
features, classes = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)

In [3]:
classes = classes.astype(numpy.int64)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    features,
    classes,
    test_size=0.15,
    random_state=42,
)

x = concrete
y = normal

### Let's use the sklearn cross-validation tool to find the best hyper parameters for our model

Grid search: It works by systematically trying out every possible combination of hyperparameters within a specified range or list of values and evaluating the model's performance for each combination using cross-validation.    

cv: sets the number of folds in cross-validation. The training data is split into 10 parts, and the model is trained and validated 10 times, each time using a different part as the validation set and the remaining parts as the training set.

n_jobs: sets the number of jobs to run in parallel. n_jobs=1 means the tasks will be run sequentially, n_jobs=-1 would use all available processors.

In [5]:
# List of hyper parameters to tune
param_grid = {
    "max_features": [None, "auto", "sqrt", "log2"],
    "min_samples_leaf": [1, 10, 100],
    "min_samples_split": [2, 10, 100],
    "max_depth": [None, 2, 4, 6, 8],
}

In [6]:
# Find best hyper parameters with cross validation
grid_search = GridSearchCV(
    ConcreteDecisionTreeClassifier(),
    param_grid,
    cv=10,
    scoring="average_precision",
    error_score="raise",
    n_jobs=1,
)
gs_results = grid_search.fit(x_train, y_train)

In [7]:
print("Best hyper parameters:", gs_results.best_params_)
print("Best score:", gs_results.best_score_)

Best hyper parameters: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 100}
Best score: 0.9302161645088886


In [8]:
# Define model with best hyper parameters
model = ConcreteDecisionTreeClassifier(
    max_features=gs_results.best_params_["max_features"],
    min_samples_leaf=gs_results.best_params_["min_samples_leaf"],
    min_samples_split=gs_results.best_params_["min_samples_split"],
    max_depth=gs_results.best_params_["max_depth"],
    n_bits=6,
)

### Let's compute some metrics on the test set.

In [9]:
# Benchmark model for concrete vs. concrete_fhe vs. normal sklearn
model, sklearn_model = model.fit_benchmark(x_train, y_train)

In [10]:
print(model)
print(sklearn_model)

DecisionTreeClassifier(min_samples_leaf=10, min_samples_split=100,
                       n_bits={'op_inputs': 6, 'op_leaves': 6},
                       random_state=22379)
DecisionTreeClassifier(min_samples_leaf=10, min_samples_split=100,
                       random_state=22379)


In [11]:
# Compute average precision on test
# pylint: disable=no-member
y_pred_concrete = model.predict_proba(x_test)[:, 1]
y_pred_sklearn = sklearn_model.predict_proba(x_test)[:, 1]

In [12]:
concrete_average_precision = average_precision_score(y_test, y_pred_concrete)
sklearn_average_precision = average_precision_score(y_test, y_pred_sklearn)

In [13]:
print(f"Concrete average precision score: {concrete_average_precision:0.2f}")
print(f"Sklearn average precision score: {sklearn_average_precision:0.2f}")

Concrete average precision score: 0.97
Sklearn average precision score: 0.95


Note that Concrete average precision score is not running in FHE here as it would be much longer. If you want to run the model in FHE you can set the argument `fhe` to `execute` in `predict_proba()`. Also, the average precision of the Concrete model may be higher which is likely due to the quantization acting as a kind of regularization which improved the test set metric. However, in general, it should be expected that quantization decreases the average precision.

In [16]:
# Confusion matrix
x_pred = model.predict(x_test)
true_negative, false_positive, false_negative, true_positive = confusion_matrix(
    y_test, x_pred, normalize="true"
).ravel()

num_samples = len(y_test)
num_spam = sum(y_test)

print(f"Number of test samples: {num_samples}")
print(f"Number of spams in test samples: {num_spam}")

print(f"True Negative (legit mail well classified) rate: {true_negative}")
print(f"False Positive (legit mail classified as spam) rate: {false_positive}")
print(f"False Negative (spam mail classified as legit) rate: {false_negative}")
print(f"True Positive (spam well classified) rate: {true_positive}")

Number of test samples: 691
Number of spams in test samples: 304
True Negative (legit mail well classified) rate: 0.9612403100775194
False Positive (legit mail classified as spam) rate: 0.03875968992248062
False Negative (spam mail classified as legit) rate: 0.14473684210526316
True Positive (spam well classified) rate: 0.8552631578947368


### FHE

In [18]:
# Compile the concrete model
circuit = model.compile(x_train)

### Generate the key

In [19]:
print(f"Generating a key for an {circuit.graph.maximum_integer_bit_width()}-bit circuit")

Generating a key for an 8-bit circuit


In [20]:
time_begin = time.time()
circuit.client.keygen(force=False)
print(f"Key generation time: {time.time() - time_begin:.2f} seconds")

Key generation time: 0.75 seconds


In [21]:
# Reduce the sample size for a faster total execution time
FHE_SAMPLES = 10
x_test_reduced = x_test[:FHE_SAMPLES]
y_pred_reduced = y_pred[:FHE_SAMPLES]
y_reference_reduced = y_test[:FHE_SAMPLES]

In [28]:
# Predict in FHE for sample size = 10
time_begin = time.time()
x_pred_fhe_reduced = model.predict(x_test_reduced, fhe="execute")
print(f"Execution time: {(time.time() - time_begin) / len(x_test):.2f} seconds per sample")

Execution time: 0.03 seconds per sample


In [29]:
# Check prediction FHE vs sklearn
print(f"Ground truth:       {y_reference_reduced}")
print(f"Prediction sklearn: {y_pred_reduced}")
print(f"Prediction FHE:     {x_pred_fhe_reduced}")

Ground truth:       [0 0 0 1 0 1 0 0 0 0]
Prediction sklearn: [0 0 0 1 0 1 0 0 0 0]
Prediction FHE:     [0 0 0 1 0 1 0 0 0 0]


In [30]:
print(
    f"{numpy.sum(x_pred_fhe_reduced==y_pred_reduced)}/"
    "10 predictions are similar between the FHE model and the clear sklearn model."
)

10/10 predictions are similar between the FHE model and the clear sklearn model.


In [ ]:
# Predict in FHE for actual sample size = 100
time_begin = time.time()
x_pred_fhe = model.predict(x_test, fhe="execute")
print(f"Execution time: {(time.time() - time_begin) / len(x_test):.2f} seconds per sample")

In [ ]:
# Check prediction FHE vs sklearn
print(f"Ground truth:       {y_reference}")
print(f"Prediction sklearn: {y_pred}")
print(f"Prediction FHE:     {x_pred_fhe}")

In [ ]:
print(
    f"{numpy.sum(x_pred_fhe_reduced==y_pred_reduced)}/"
    "100 predictions are similar between the FHE model and the clear sklearn model."
)